# 데이터 불러오기

In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
base_path = './preprocessed data/'

train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
train_quality = pd.read_csv(base_path + 'new_train_quality.csv')
train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])
test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

# feature engineering

default = model_nm, fwver, errtype, counts
1. multiple_model + multiple_fwver + errtype_num + errtype_weight + errcode_weight + quality_sum

In [6]:
df1 = train_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0


In [7]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,1
...,...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0,1
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0,1


In [8]:
# errtype, errcode에 대한 가중치 column추가 (errtype_weight, errcode_weight)

# 1. errtype_weight
errtype_df  = train_err[['user_id', 'errtype']]
errtype_df = errtype_df.drop_duplicates(subset=None, keep='first')
errtype_df = errtype_df.groupby('errtype')['user_id'].unique().to_frame().reset_index()
errtype_df['counts'] = 0
errtype_df['errtype_weight'] = 0

for i in range(len(errtype_df.index)):
    errtype_df['counts'][i] = len(list(train_problem[train_problem['user_id'].isin(errtype_df.user_id[i])].user_id.unique()))
    
for i in range(len(errtype_df.index)):
        errtype_df['errtype_weight'][i] = round(errtype_df['counts'][i]/len(errtype_df.user_id[i]), 2)
errtype_df.drop(columns=['user_id', 'counts'], inplace=True)

errtype_df.head()

,errtype,errtype_weight
0,1,0.25
1,2,0.43
2,3,0.28
3,4,0.33
4,5,0.37


In [9]:
# 2. errcode_weight
errcode_df  = train_err[['user_id', 'errcode']]
errcode_df = errcode_df.drop_duplicates(subset=None, keep='first')
errcode_df = errcode_df.groupby('errcode')['user_id'].unique().to_frame().reset_index()
errcode_df['counts'] = 0
errcode_df['errcode_weight'] = 0

for i in range(len(errcode_df.index)):
    errcode_df['counts'][i] = len(list(train_problem[train_problem['user_id'].isin(errcode_df.user_id[i])].user_id.unique()))
    
for i in range(len(errcode_df.index)):
        errcode_df['errcode_weight'][i] = round(errcode_df['counts'][i]/len(errcode_df.user_id[i]), 2)
errcode_df.drop(columns=['user_id', 'counts'], inplace=True)

errcode_df.head()

,errcode,errcode_weight
0,-269,0.64
1,-270,0.36
2,0,0.34
3,0001,0.50
4,1,0.33


In [10]:
df1 = pd.merge(df1, errtype_df, how='left', on='errtype')
df1 = pd.merge(df1, errcode_df, how='left', on='errcode')
df1.head(10)

,user_id,time,model_nm,fwver,errtype,errcode,counts,errtype_weight,errcode_weight
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,1,0.34,0.33
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,1,0.34,0.33
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,1,0.34,0.33
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,1,0.34,0.33
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,1,0.33,0.34
5,10000,2020-11-01 05:05:16,model_3,05.15.2138,26,1,1,0.36,0.33
6,10000,2020-11-01 05:05:22,model_3,05.15.2138,4,0,1,0.33,0.34
7,10000,2020-11-01 06:09:03,model_3,05.15.2138,15,1,1,0.34,0.33
8,10000,2020-11-01 17:47:54,model_3,05.15.2138,16,1,1,0.34,0.33
9,10000,2020-11-01 17:47:56,model_3,05.15.2138,4,0,1,0.33,0.34


In [11]:
# multiple_model
model_df =  train_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
10000,0
10001,0
10002,0
10003,0
10004,0
10005,0
10006,1


In [12]:
# multiple_fwver
fwver_df =  train_err[['user_id', 'fwver']]
fwver_df = fwver_df.drop_duplicates(subset=None, keep='first')

fwver_num = fwver_df.user_id.value_counts(sort=False)
user_has_multiple_fwvers = fwver_num > 1

user_has_multiple_fwvers = user_has_multiple_fwvers.astype(int)
user_has_multiple_fwvers = user_has_multiple_fwvers.to_frame(name='multiple_fwver')
user_has_multiple_fwvers.head(7)

,multiple_fwver
10000,0
10001,1
10002,0
10003,1
10004,1
10005,1
10006,1


In [13]:
# errtype_num
errtype_df = train_err[['user_id', 'errtype']]
errtype_df = errtype_df.user_id.value_counts(sort=False).to_frame(name='errtype_num')
errtype_df.head()

,errtype_num
10000,316
10001,2345
10002,305
10003,297
10004,777


In [14]:
# quality_sum
quality_df = pd.DataFrame({'user_id': np.arange(10000, 25000)})

for i in train_quality.columns[3:]:
    quality_group_df = train_quality.groupby('user_id')[i].sum().to_frame().reset_index()
    quality_df = pd.merge(quality_df, quality_group_df, how='left', on='user_id')
    quality_df = quality_df.fillna(0)
    
# 모두 0값인 quality_3, quality_4 drop
quality_df.set_index(quality_df['user_id'], inplace=True)
quality_df = quality_df.drop(columns=['user_id', 'quality_3', 'quality_4'])

quality_df.head()

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,,
10000,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,1.0,-1.0,0.0,30.0,45.0,96.0,0.0,2.0,95.0,-1.0,0.0
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,-1.0,-1.0,-1.0,3.0,86.0,174.0,0.0,0.0,14.0,-1.0,0.0


Train set 만들기

In [15]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,counts,errtype_weight,errcode_weight,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_03.11.1167,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3553,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1750,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.33.1261,fwver_04.73.2237,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.2138,fwver_05.15.3104,fwver_05.66.3237,fwver_05.66.3571,fwver_10,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,1,0.33,0.34,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367996,24999,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15367997,24999,1,0.34,0.33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15367998,24999,1,0.33,0.34,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15367999,24999,1,0.33,0.34,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
# 2. errtype_weight, errcode_weight를 user_id별로 다시 가중치 적용

# 2-1. errtype_weight
errtype_weight_df = df1[['user_id', 'errtype_weight']]
errtype_weight_df = errtype_weight_df.groupby('user_id')['errtype_weight'].sum().to_frame(name='errtype_weight')
total_sum = errtype_weight_df.errtype_weight.sum()
errtype_weight_df.errtype_weight = errtype_weight_df.errtype_weight / total_sum
errtype_weight_df

,errtype_weight
user_id,
10000,1.864538e-05
10001,1.549039e-04
10002,1.786448e-05
10003,1.858874e-05
10004,4.892524e-05
...,...
24995,1.217510e-05
24996,2.969119e-07
24997,5.098132e-05


In [17]:
# 2-2. errcode_weight
errcode_weight_df = df1[['user_id', 'errcode_weight']]
errcode_weight_df = errcode_weight_df.groupby('user_id')['errcode_weight'].sum().to_frame(name='errcode_weight')
total_sum = errcode_weight_df.errcode_weight.sum()
errcode_weight_df.errcode_weight = errcode_weight_df.errcode_weight / total_sum
errcode_weight_df

,errcode_weight
user_id,
10000,1.897493e-05
10001,1.522731e-04
10002,1.835360e-05
10003,1.807085e-05
10004,4.840085e-05
...,...
24995,1.210967e-05
24996,2.755471e-07
24997,4.983081e-05


In [18]:
# 모든 column 합쳐주기
df1 = df1.groupby('user_id').sum()
df1.drop(columns=['errtype_weight', 'errcode_weight'], inplace=True)
dfs = [df1, user_has_multiple_models, user_has_multiple_fwvers, errtype_df, errtype_weight_df, errcode_weight_df, quality_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_03.11.1167,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3553,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1750,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.33.1261,fwver_04.73.2237,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.2138,fwver_05.15.3104,fwver_05.66.3237,fwver_05.66.3571,fwver_10,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_model,multiple_fwver,errtype_num,errtype_weight,errcode_weight,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,316,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,316,1.864538e-05,1.897493e-05,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
10001,2345,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,368.0,1977.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,8.0,143.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0,1,2345,1.549039e-04,1.522731e-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,305,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,131.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,305,1.786448e-05,1.835360e-05,1.0,-1.0,0.0,30.0,45.0,96.0,0.0,2.0,95.0,-1.0,0.0
10003,297,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,217.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,45.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,57.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0,1,297,1.858874e-05,1.807085e-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,777,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,645.0,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0,1,777,4.892524e-05,4.840085e-05,-1.0,-1.0,-1.0,3.0,86.0,174.0,0.0,0.0,14.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [19]:
# 중복정보가 있는 column 제거
def remove_collinearity(X, threshold):
    """
    X : feature matrix
    threshold : 다중공선성을 제거할 column을 고르는 기준 값. [0, 1]
    """
    
    corr = X.corr()
    candidate_cols = []
    
    for x in corr.iterrows():
        idx, row = x[0], x[1] # decoupling tuple
        if idx in candidate_cols:
            continue
        candidates = row[row > threshold].index[1:]

        # 자기 자신을 제외하고 threshold를 넘는 column이 있다면 제거 할 column이다.
        if len(candidates) != 0:
            for col in candidates:
                candidate_cols.append(col)           
    
    return candidate_cols

In [20]:
correlated_features = remove_collinearity(dfs, 0.7)
print(correlated_features)
dfs = dfs.drop(columns=correlated_features)
len(dfs.columns)

['errtype_4', 'errtype_num', 'errtype_weight', 'errcode_weight', 'fwver_04.22.1750', 'fwver_04.22.1778', 'fwver_04.16.3553', 'errtype_22', 'errtype_23', 'fwver_04.33.1185', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_04.82.1778', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_10', 'errtype_11', 'errtype_12', 'errtype_7', 'errtype_16', 'errtype_20', 'errtype_20', 'errtype_21', 'errtype_35', 'errtype_37', 'multiple_fwver', 'quality_2', 'quality_9', 'quality_10', 'quality_7']


70

In [21]:
dfs.index = dfs.index-10000

In [22]:
dfs2 = dfs.copy()

In [23]:
dfs2 = dfs2.to_numpy()
dfs2

array([[ 3.160e+02,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 2.345e+03,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 3.050e+02,  0.000e+00,  0.000e+00, ...,  0.000e+00, -1.000e+00,
         0.000e+00],
       ...,
       [ 8.180e+02,  8.180e+02,  0.000e+00, ...,  0.000e+00, -1.000e+00,
         0.000e+00],
       [ 1.440e+02,  1.440e+02,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 5.660e+02,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00]])

In [24]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [25]:
train_x = dfs2
train_y = problem

print(train_x.shape, train_y.shape)

(15000, 70) (15000,)


In [26]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002435 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4874
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[15]	valid_0's auc: 0.806957	valid_0's pr_auc: 0.809419
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001633 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4954
[LightGBM] [Info] Number of data points in the train set: 12000, number of used featur

In [27]:
print(np.mean(auc_scores))

0.7973963144780454


In [28]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [29]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,problem
0,316.0,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,1.0,10.0,59.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
1,2345.0,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,1.0,0.0,0.0,1.0,8.0,143.0,0.0,4.0,1.0,5.0,1.0,22.0,0.0,250.0,0.0,10.0,18.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,305.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,4.0,52.0,0.0,0.0,0.0,2.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,30.0,45.0,0.0,-1.0,0.0,0.0
3,297.0,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,45.0,1.0,0.0,0.0,0.0,0.0,57.0,0.0,65.0,0.0,8.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,777.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,5.0,143.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,176.0,0.0,16.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,0.0,-1.0,-1.0,3.0,86.0,0.0,-1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,191.0,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,8.0,26.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0,-2.0,-2.0,3.0,-2.0,0.0,-2.0,0.0,0.0
14996,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,818.0,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,3.0,1.0,181.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,223.0,0.0,16.0,12.0,1.0,0.0,0.0,58.0,7.0,5.0,0.0,-1.0,-1.0,16.0,-1.0,0.0,-1.0,0.0,1.0
14998,144.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,51.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,14.0,0.0,15.0,2.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [37]:
clf = setup(data = train, target = "problem") 

,Description,Value
0,session_id,8607
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 71)"
5,Missing Values,False
6,Numeric Features,62
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


In [38]:
best_5 = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7916,0.8123,0.4777,0.8103,0.6008,0.4722,0.5026,0.2780
rf,Random Forest Classifier,0.7905,0.8104,0.4997,0.7847,0.6102,0.4764,0.4993,0.1770
lightgbm,Light Gradient Boosting Machine,0.7899,0.8087,0.5101,0.7732,0.6145,0.4783,0.4980,0.0500
et,Extra Trees Classifier,0.7873,0.8069,0.4910,0.7799,0.6024,0.4674,0.4908,0.1820
ada,Ada Boost Classifier,0.7821,0.8032,0.4837,0.7673,0.5931,0.4545,0.4773,0.0890
lda,Linear Discriminant Analysis,0.7566,0.7666,0.3242,0.8324,0.4659,0.3461,0.4108,0.0370
nb,Naive Bayes,0.6815,0.6656,0.1612,0.5491,0.2486,0.1183,0.1539,0.0160
dt,Decision Tree Classifier,0.7018,0.6653,0.5589,0.5450,0.5517,0.3284,0.3285,0.0310
knn,K Neighbors Classifier,0.6884,0.6630,0.3646,0.5388,0.4347,0.2305,0.2390,0.2120
qda,Quadratic Discriminant Analysis,0.4399,0.6549,0.8626,0.3955,0.5098,0.0932,0.1023,0.0240


In [39]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7916,0.8123,0.4777,0.8103,0.6008,0.4722,0.5026,0.2970
rf,Random Forest Classifier,0.7905,0.8104,0.4997,0.7847,0.6102,0.4764,0.4993,0.1740
lightgbm,Light Gradient Boosting Machine,0.7899,0.8087,0.5101,0.7732,0.6145,0.4783,0.4980,0.0510
et,Extra Trees Classifier,0.7873,0.8069,0.4910,0.7799,0.6024,0.4674,0.4908,0.1940
ada,Ada Boost Classifier,0.7821,0.8032,0.4837,0.7673,0.5931,0.4545,0.4773,0.0820
lda,Linear Discriminant Analysis,0.7566,0.7666,0.3242,0.8324,0.4659,0.3461,0.4108,0.0430
nb,Naive Bayes,0.6815,0.6656,0.1612,0.5491,0.2486,0.1183,0.1539,0.0160
dt,Decision Tree Classifier,0.7018,0.6653,0.5589,0.5450,0.5517,0.3284,0.3285,0.0320
knn,K Neighbors Classifier,0.6884,0.6630,0.3646,0.5388,0.4347,0.2305,0.2390,0.0850
qda,Quadratic Discriminant Analysis,0.4399,0.6549,0.8626,0.3955,0.5098,0.0932,0.1023,0.0240


In [40]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7881,0.8260,0.4848,0.7877,0.6002,0.4669,0.4924
1,0.7890,0.8148,0.4725,0.8049,0.5954,0.4655,0.4958
2,0.7905,0.8165,0.5014,0.7828,0.6113,0.4772,0.4993
3,0.8019,0.8238,0.5116,0.8171,0.6292,0.5036,0.5294
4,0.7832,0.8004,0.4761,0.7773,0.5905,0.4544,0.4797
Mean,0.7906,0.8163,0.4893,0.7940,0.6053,0.4735,0.4993
Std,0.0062,0.0090,0.0150,0.0148,0.0138,0.0167,0.0164


In [41]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7854,0.8085,0.4878,0.8157,0.6105,0.4750,0.5049


In [42]:
final_model = finalize_model(blended)

In [45]:
# predictions = predict_model(final_model, data = test)